## Now that we have the basic MCLP working with SCIP (pyscipopt), let's try to formulate and solve the basic C-MCLP

#original code from the MCLP (converted from FLP)

```python

def flpo(I,J,d,M,c):
    """flp -- model for the capacitated facility location problem
    Parameters:
        - I: set of customers
        - J: set of facilities
        - d[i]: demand for customer i
        - M[j]: capacity of facility j
        - c[i,j]: unit cost of servicing demand point i from facility j
    Returns a model, ready to be solved.
    """

    model = Model("flp")

    x,y = {},{}
    for j in J: # FACILITIES
        x[j] = model.addVar(vtype="B", name="y(%s)"%j) 
    for i in I: # CUSTOMERS
        y[i] = model.addVar(vtype="C", name="x(%s)"%i)


    N= {1:[], 2:[2], 3:[2], 4:[1,3], 5:[3], 6:[2,3], 7:[1,2,3]}
    
    #ADDED
    for i in I:
        model.addCons(quicksum(x[j] for j in N[i])+y[i] >= 1)
    
    for i in I:
        model.addCons(y[i] >= 0)
    
    P = 2
    model.addCons(quicksum(x[j] for j in J) == P)
    
    model.setObjective(
        quicksum(y[i]*d[i] for i in I),
        "minimize")
    model.data = x,y

    return model

def make_datao():
    I,d = multidict({1:12, 2:5, 3:6, 4:3, 5:9, 6:1, 7:10})            # demand
    J,M = multidict({1:[500], 2:[500], 3:[500]}) # capacity, fixed costs
    c = {(1,1):4,  (1,2):6,  (1,3):9,    # distance cost to get from demand point i to 
         (2,1):5,  (2,2):0.7,  (2,3):7,   # potl facility site j (i,j):distance
         (3,1):6,  (3,2):3,  (3,3):4,
         (4,1):0.5,  (4,2):5,  (4,3):3,
         (5,1):10, (5,2):8,  (5,3):0.86,
         (6,1):9,  (6,2):0.8,  (6,3):0.7,
         (7,1):2.5,  (7,2):2.5,  (7,3):1.3,
         }
    return I,J,d,M,c


if __name__ == "__main__":
    I,J,d,M,c = make_datao()
    model = flpo(I,J,d,M,c)
    model.hideOutput(quiet=False)
    model.optimize()

    EPS = 1.e-6
    x,y = model.data
    uncovered_customers = [i for i in y if model.getVal(y[i]) > EPS]
    # ADDED:
    # remember, we need to switch this b/c y==1 means its NOT covered in our model
    covered_customers = [i for i in y if i not in uncovered_customers]
    facilities = [j for j in x if model.getVal(x[j]) > EPS]

    print("Optimal value:", model.getObjVal())
    print("Facilities at nodes:", facilities)
    print("Customers Covered:", covered_customers)


```


## ---------------------------------------------------------------------------

In [ ]:
def make_datao():
    I,d = multidict({1:12, 2:5, 3:6, 4:3, 5:9, 6:1, 7:10})            # demand
    J,M = multidict({1:[500], 2:[500], 3:[500]}) # capacity, fixed costs
    c = {(1,1):4,  (1,2):6,  (1,3):9,    # distance cost to get from demand point i to 
         (2,1):5,  (2,2):0.7,  (2,3):7,   # potl facility site j (i,j):distance
         (3,1):6,  (3,2):3,  (3,3):4,
         (4,1):0.5,  (4,2):5,  (4,3):3,
         (5,1):10, (5,2):8,  (5,3):0.86,
         (6,1):9,  (6,2):0.8,  (6,3):0.7,
         (7,1):2.5,  (7,2):2.5,  (7,3):1.3,
         }
    return I,J,d,M,c

In [ ]:
def flpo(I,J,d,M,c):
    """flp -- model for the capacitated facility location problem
    Parameters:
        - I: set of customers
        - J: set of facilities
        - d[i]: demand for customer i
        - M[j]: capacity of facility j
        - c[i,j]: unit cost of servicing demand point i from facility j
    Returns a model, ready to be solved.
    """

    model = Model("flp")

    x,y,e = {},{}           # (x[j]: 1 if @ j, 0 if not)
    for j in J: # FACILITIES (e[j]: 0 if @ j, 1 if not)
        x[j] = model.addVar(vtype="B", name="y(%s)"%j)
        e[j] = model.addVar(vtype="B", name="e(%s)"%j)
        
    for i in I: # CUSTOMERS (0 if i covered, 1 if not)
        y[i] = model.addVar(vtype="C", name="x(%s)"%i)
    


    N= {1:[], 2:[2], 3:[2], 4:[1,3], 5:[3], 6:[2,3], 7:[1,2,3]}

    #ADDED
    for i in I:
        model.addCons(quicksum(x[j]+ e[j] for j in N[i]) >= 1)
    
    for i in I:
        model.addCons(quicksum(x[j] for j in N[i])+y[i] >= 1)

    for i in I:
        model.addCons(y[i] >= 0)

    P = 2
    model.addCons(quicksum(x[j] for j in J) == P)

    model.setObjective(
        quicksum(y[i]*d[i] for i in I),
        "minimize")
    model.data = x,y

    return model


In [ ]:
if __name__ == "__main__":
    I,J,d,M,c = make_datao()
    model = flpo(I,J,d,M,c)
    model.hideOutput(quiet=False)
    model.optimize()

    EPS = 1.e-6
    x,y = model.data
    uncovered_customers = [i for i in y if model.getVal(y[i]) > EPS]
    # ADDED:
    # remember, we need to switch this b/c y==1 means its NOT covered in our model
    covered_customers = [i for i in y if i not in uncovered_customers]
    facilities = [j for j in x if model.getVal(x[j]) > EPS]

    print("Optimal value:", model.getObjVal())
    print("Facilities at nodes:", facilities)
    print("Customers Covered:", covered_customers)

In [3]:
import pyscipopt

In [ ]:
pyscipopt.scip.